Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
import operator

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

In [11]:
batch_size = 128

def create_sgd_relu_graph(nb_hidden, l2_reg=False):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, nb_hidden]))
        biases_1 = tf.Variable(tf.zeros([nb_hidden]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
        # output layer
        weights_2 = tf.Variable(
            tf.truncated_normal([nb_hidden, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights_2) + biases_2

        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        
        if l2_reg:
            loss += 5e-4 * reduce(operator.add, [tf.nn.l2_loss(w) for w in (weights_1, weights_2)])
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)
        
    return (graph, optimizer, loss,
            { 'dataset': tf_train_dataset, 'labels': tf_train_labels },
            { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction })

In [16]:
num_steps = 3001

def run_sgd_relu_computation(nb_hidden, l2_reg=False):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph(nb_hidden, l2_reg)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))

In [17]:
for i in (10, 100, 500, 1000, 5000, 10000):
    %time run_sgd_relu_computation(i, True)


Initialized for 10 hidden relu
Minibatch loss at step 0: 24.631552
Minibatch accuracy: 14.1%
Validation accuracy: 22.9%
Minibatch loss at step 500: 2.135665
Minibatch accuracy: 70.3%
Validation accuracy: 72.5%
Minibatch loss at step 1000: 1.768020
Minibatch accuracy: 75.8%
Validation accuracy: 73.3%
Minibatch loss at step 1500: 1.386657
Minibatch accuracy: 78.9%
Validation accuracy: 77.9%
Minibatch loss at step 2000: 1.231885
Minibatch accuracy: 78.9%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 0.930158
Minibatch accuracy: 83.6%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 1.036868
Minibatch accuracy: 78.9%
Validation accuracy: 81.4%
Test accuracy: 88.9%
CPU times: user 9.2 s, sys: 5.29 s, total: 14.5 s
Wall time: 4.83 s

Initialized for 100 hidden relu
Minibatch loss at step 0: 121.948509
Minibatch accuracy: 17.2%
Validation accuracy: 29.2%
Minibatch loss at step 500: 12.825439
Minibatch accuracy: 68.0%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 10

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
num_steps = 3001

def run_sgd_relu_computation_overfit(nb_hidden, l2_reg=False):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph(nb_hidden, l2_reg)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = batch_size * np.random.choice(10)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))

In [19]:
for i in (10, 100, 500, 1000, 5000, 10000):
    %time run_sgd_relu_computation_overfit(i, True)


Initialized for 10 hidden relu
Minibatch loss at step 0: 35.844971
Minibatch accuracy: 7.8%
Validation accuracy: 14.5%
Minibatch loss at step 500: 1.690555
Minibatch accuracy: 81.2%
Validation accuracy: 66.8%
Minibatch loss at step 1000: 1.139535
Minibatch accuracy: 92.2%
Validation accuracy: 70.0%
Minibatch loss at step 1500: 0.835842
Minibatch accuracy: 96.1%
Validation accuracy: 69.7%
Minibatch loss at step 2000: 0.681152
Minibatch accuracy: 96.9%
Validation accuracy: 70.8%
Minibatch loss at step 2500: 0.624337
Minibatch accuracy: 94.5%
Validation accuracy: 71.3%
Minibatch loss at step 3000: 0.507316
Minibatch accuracy: 96.1%
Validation accuracy: 71.8%
Test accuracy: 79.7%
CPU times: user 9.57 s, sys: 4.85 s, total: 14.4 s
Wall time: 4.64 s

Initialized for 100 hidden relu
Minibatch loss at step 0: 123.796310
Minibatch accuracy: 8.6%
Validation accuracy: 29.4%
Minibatch loss at step 500: 11.909293
Minibatch accuracy: 100.0%
Validation accuracy: 71.4%
Minibatch loss at step 1000: 9.

You have great (perfect) minibatch accuracy, but:
* bad validation accuracy
* bad test accuracy

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [23]:
batch_size = 128

def create_sgd_relu_graph(nb_hidden, l2_reg=False, dropout=False):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, nb_hidden]))
        biases_1 = tf.Variable(tf.zeros([nb_hidden]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        if dropout:
            hidden = tf.nn.dropout(hidden, 0.5)
        
        # output layer
        weights_2 = tf.Variable(
            tf.truncated_normal([nb_hidden, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights_2) + biases_2

        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        if l2_reg:
            loss += 5e-4 * reduce(operator.add, [tf.nn.l2_loss(w) for w in (weights_1, weights_2)])
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)
        
    return (graph, optimizer, loss,
            { 'dataset': tf_train_dataset, 'labels': tf_train_labels },
            { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction })

In [24]:
num_steps = 3001

def run_sgd_relu_computation_overfit(nb_hidden, l2_reg=False, dropout=False):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph(nb_hidden, l2_reg, dropout)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = batch_size * np.random.choice(10)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))

In [25]:
for i in (10, 100, 500, 1000, 5000, 10000):
    %time run_sgd_relu_computation_overfit(i, True, True)


Initialized for 10 hidden relu
Minibatch loss at step 0: 42.585793
Minibatch accuracy: 3.1%
Validation accuracy: 15.1%
Minibatch loss at step 500: 2.857259
Minibatch accuracy: 32.8%
Validation accuracy: 49.5%
Minibatch loss at step 1000: 2.541906
Minibatch accuracy: 34.4%
Validation accuracy: 58.4%
Minibatch loss at step 1500: 2.194010
Minibatch accuracy: 42.2%
Validation accuracy: 62.7%
Minibatch loss at step 2000: 1.912450
Minibatch accuracy: 42.2%
Validation accuracy: 63.0%
Minibatch loss at step 2500: 1.734928
Minibatch accuracy: 46.9%
Validation accuracy: 65.7%
Minibatch loss at step 3000: 1.620522
Minibatch accuracy: 53.1%
Validation accuracy: 65.2%
Test accuracy: 72.0%
CPU times: user 10.6 s, sys: 6.3 s, total: 16.9 s
Wall time: 5.71 s

Initialized for 100 hidden relu
Minibatch loss at step 0: 177.008591
Minibatch accuracy: 7.8%
Validation accuracy: 25.5%
Minibatch loss at step 500: 12.040442
Minibatch accuracy: 88.3%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 9.36

Validation accuracy is a little bit better (+0.9%).

Test accuracy is a little bit better (+1.1%).

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Try a 1 hidden-layer NN with L2 regularization and dropout

In [26]:
num_steps = 3001

def run_sgd_relu_computation(nb_hidden, l2_reg=False, dropout=False):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph(nb_hidden, l2_reg, dropout)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))

In [27]:
for i in (10, 100, 500, 1000, 5000, 10000):
    %time run_sgd_relu_computation(i, True, True)


Initialized for 10 hidden relu
Minibatch loss at step 0: 59.175949
Minibatch accuracy: 8.6%
Validation accuracy: 12.7%
Minibatch loss at step 500: 3.069124
Minibatch accuracy: 21.9%
Validation accuracy: 49.4%
Minibatch loss at step 1000: 2.830744
Minibatch accuracy: 36.7%
Validation accuracy: 52.2%
Minibatch loss at step 1500: 2.383336
Minibatch accuracy: 36.7%
Validation accuracy: 56.8%
Minibatch loss at step 2000: 2.170527
Minibatch accuracy: 39.8%
Validation accuracy: 62.5%
Minibatch loss at step 2500: 1.792706
Minibatch accuracy: 50.8%
Validation accuracy: 73.5%
Minibatch loss at step 3000: 1.817116
Minibatch accuracy: 42.2%
Validation accuracy: 74.4%
Test accuracy: 81.6%
CPU times: user 10.7 s, sys: 6.22 s, total: 16.9 s
Wall time: 5.58 s

Initialized for 100 hidden relu
Minibatch loss at step 0: 157.614868
Minibatch accuracy: 6.2%
Validation accuracy: 28.2%
Minibatch loss at step 500: 13.207933
Minibatch accuracy: 57.0%
Validation accuracy: 73.1%
Minibatch loss at step 1000: 10.

Accuracy is worse with dropout (on a 1 hidden layer NN).

Implement a NN with 2 hidden layers

In [74]:
batch_size = 128

def create_sgd_relu_graph_2(nb_hidden1, nb_hidden2, l2_reg=False, dropout=False):
    hidden_1_stddev = np.sqrt(2.0/nb_hidden1) 
    hidden_2_stddev = np.sqrt(2.0/nb_hidden1)
    
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size, image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer n°1
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, nb_hidden1], stddev=hidden_1_stddev))
        biases_1 = tf.Variable(tf.zeros([nb_hidden1]))
        hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        if dropout:
            hidden_1 = tf.nn.dropout(hidden_1, 0.5)
            
        # hidden layer n°2
        weights_2 = tf.Variable(
            tf.truncated_normal([nb_hidden1, nb_hidden2], stddev=hidden_2_stddev))
        biases_2 = tf.Variable(tf.zeros([nb_hidden2]))
        hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_2) + biases_2)
#        if dropout:
#            hidden_2 = tf.nn.dropout(hidden_2, 0.5)
        
        # output layer
        weights_3 = tf.Variable(
            tf.truncated_normal([nb_hidden2, num_labels]))
        biases_3 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden_2, weights_3) + biases_3

        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        if l2_reg:
            loss += 5e-4 * reduce(operator.add, [tf.nn.l2_loss(w) for w in (weights_1, weights_2, weights_3)])
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            hid1 = tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1)
            hid2 = tf.nn.relu(tf.matmul(hid1, weights_2) + biases_2)
            return tf.nn.softmax(tf.matmul(hid2, weights_3) + biases_3)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)
        
    return (graph, optimizer, loss,
            { 'dataset': tf_train_dataset, 'labels': tf_train_labels },
            { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction })

In [75]:
num_steps = 6001

def run_sgd_relu_computation_2(nb_hidden1, nb_hidden2, l2_reg=False, dropout=False):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph_2(nb_hidden1, nb_hidden2, l2_reg, dropout)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} and {1} hidden relu'.format(nb_hidden1, nb_hidden2))
        print('L2 regularization:', l2_reg)
        print('Dropout:', dropout)
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))

In [76]:
%time run_sgd_relu_computation_2(500, 50, False, False)
%time run_sgd_relu_computation_2(500, 50, False, True)
%time run_sgd_relu_computation_2(500, 50, True, False)
%time run_sgd_relu_computation_2(500, 50, True, True)


Initialized for 500 and 50 hidden relu
L2 regularization: False
Dropout: False
Minibatch loss at step 0: 4.700445
Minibatch accuracy: 12.5%
Validation accuracy: 15.4%
Minibatch loss at step 500: 1.399379
Minibatch accuracy: 53.9%
Validation accuracy: 49.2%
Minibatch loss at step 1000: 1.190712
Minibatch accuracy: 57.8%
Validation accuracy: 59.4%
Minibatch loss at step 1500: 1.156577
Minibatch accuracy: 61.7%
Validation accuracy: 64.5%
Minibatch loss at step 2000: 0.976141
Minibatch accuracy: 64.1%
Validation accuracy: 63.5%
Minibatch loss at step 2500: 0.847928
Minibatch accuracy: 72.7%
Validation accuracy: 68.7%
Minibatch loss at step 3000: 0.888021
Minibatch accuracy: 71.9%
Validation accuracy: 72.4%
Minibatch loss at step 3500: 0.699344
Minibatch accuracy: 74.2%
Validation accuracy: 73.8%
Minibatch loss at step 4000: 0.844791
Minibatch accuracy: 69.5%
Validation accuracy: 75.2%
Minibatch loss at step 4500: 0.561439
Minibatch accuracy: 81.2%
Validation accuracy: 76.2%
Minibatch loss

Try with a higher number of nodes in the 1st hidden layer

In [77]:
%time run_sgd_relu_computation_2(1000, 50, True, False)


Initialized for 1000 and 50 hidden relu
L2 regularization: True
Dropout: False
Minibatch loss at step 0: 4.052834
Minibatch accuracy: 7.8%
Validation accuracy: 23.7%
Minibatch loss at step 500: 0.886473
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.962920
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 0.688535
Minibatch accuracy: 89.1%
Validation accuracy: 86.5%
Minibatch loss at step 2000: 0.750334
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 0.478417
Minibatch accuracy: 93.0%
Validation accuracy: 87.2%
Minibatch loss at step 3000: 0.578756
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
Minibatch loss at step 3500: 0.488876
Minibatch accuracy: 91.4%
Validation accuracy: 87.7%
Minibatch loss at step 4000: 0.620746
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 4500: 0.473933
Minibatch accuracy: 89.1%
Validation accuracy: 87.4%
Minibatch loss 

In [78]:
%time run_sgd_relu_computation_2(10000, 50, True, False)


Initialized for 10000 and 50 hidden relu
L2 regularization: True
Dropout: False
Minibatch loss at step 0: 2.782894
Minibatch accuracy: 14.8%
Validation accuracy: 25.0%
Minibatch loss at step 500: 1.503133
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 1.292612
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 1.006169
Minibatch accuracy: 89.1%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.896536
Minibatch accuracy: 83.6%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 0.695264
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 0.694336
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 0.568048
Minibatch accuracy: 90.6%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.759542
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 4500: 0.520910
Minibatch accuracy: 90.6%
Validation accuracy: 86.4%
Minibatch los

It's better. (+1%)

Try with a higher number of steps

In [79]:
num_steps = 25001
%time run_sgd_relu_computation_2(1000, 50, True, False)
%time run_sgd_relu_computation_2(1000, 50, True, True)


Initialized for 1000 and 50 hidden relu
L2 regularization: True
Dropout: False
Minibatch loss at step 0: 3.890655
Minibatch accuracy: 9.4%
Validation accuracy: 12.5%
Minibatch loss at step 500: 1.034017
Minibatch accuracy: 85.9%
Validation accuracy: 82.2%
Minibatch loss at step 1000: 1.029198
Minibatch accuracy: 83.6%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 0.726850
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.666156
Minibatch accuracy: 83.6%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 0.560356
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 3000: 0.621380
Minibatch accuracy: 86.7%
Validation accuracy: 87.0%
Minibatch loss at step 3500: 0.520857
Minibatch accuracy: 89.8%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.675250
Minibatch accuracy: 85.2%
Validation accuracy: 87.0%
Minibatch loss at step 4500: 0.502884
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Minibatch loss 

No improvement